In [1]:
import boto3
import configparser
import psycopg2
from sql_queries import create_table_queries, drop_table_queries


In [17]:
drop_table_queries = drop_table_queries[2:]

In [18]:
drop_table_queries

['DROP TABLE IF EXISTS staging_events;',
 'DROP TABLE IF EXISTS staging_songs;',
 'DROP TABLE IF EXISTS songplay;',
 'DROP TABLE IF EXISTS users;',
 'DROP TABLE IF EXISTS songs;',
 'DROP TABLE IF EXISTS artists;',
 'DROP TABLE IF EXISTS time;']

In [4]:
create_table_queries = create_table_queries[2:]

In [22]:
create_table_queries

['\nCREATE TABLE staging_events \n(\n    id INTEGER IDENTITY(0,1),\n    artist VARCHAR(MAX),\n    auth VARCHAR(15),\n    firstName VARCHAR(255),\n    gender VARCHAR(5),\n    itemInSession INTEGER,\n    lastName VARCHAR(255),\n    length DOUBLE PRECISION,\n    level VARCHAR(50),\n    location VARCHAR(MAX),\n    method VARCHAR(6),\n    page VARCHAR(MAX),\n    registration DOUBLE PRECISION,\n    sessionId DOUBLE PRECISION,\n    song VARCHAR(MAX),\n    status numeric(3,0),\n    ts TIMESTAMP,\n    userAgent VARCHAR(MAX),\n    userId VARCHAR(255)\n);\n',
 '\nCREATE TABLE staging_songs \n(\n    num_songs INTEGER,\n    artist_id VARCHAR(18),\n    artist_latitude DOUBLE PRECISION,\n    artist_longitude DOUBLE PRECISION,\n    artist_location VARCHAR(MAX),\n    artist_name VARCHAR(MAX),\n    song_id VARCHAR(18),\n    title VARCHAR(MAX),\n    duration DOUBLE PRECISION,\n    year numeric(4,0)\n);\n',
 '\nCREATE TABLE songplay \n(\n    songplay_id INTEGER NOT NULL,\n    start_time TIMESTAMP,\n    us

In [28]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [2]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

HOST = config.get("CLUSTER", "HOST")
DBNAME = config.get("CLUSTER", "DB_NAME")
USER = config.get("CLUSTER", "DB_USER")
PASSWORD = config.get("CLUSTER", "DB_PASSWORD")
PORT = config.get("CLUSTER", "DB_PORT")
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(HOST, DBNAME, USER, PASSWORD, PORT))
cur = conn.cursor()

In [30]:
drop_tables(cur, conn)

In [31]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [32]:
create_tables(cur, conn)

In [11]:
conn.close()

In [3]:
from sql_queries import copy_table_queries

In [4]:
copy_table_queries

["\nCOPY staging_events from 's3://udacity-dend/log_data'\ncredentials 'aws_iam_role=arn:aws:iam::999967103006:role/dwhRole'\nregion 'us-west-2'\nJSON 's3://udacity-dend/log_json_path.json'\nTIMEFORMAT as 'epochmillisecs';\n",
 "\nCOPY staging_songs from 's3://udacity-dend/song_data'\ncredentials 'aws_iam_role=arn:aws:iam::999967103006:role/dwhRole'\nregion 'us-west-2'\njson 'auto';\n"]

In [5]:
def load_staging_tables(cur, conn):
    for query in copy_table_queries:
        cur.execute(query)
        conn.commit()

In [6]:
load_staging_tables(cur, conn)